In [47]:
#Import libs 
import pandas as pd

In [48]:
#Read in train and test data 
train_data = pd.read_csv('titanic_train_data.csv')
test_data = pd.read_csv('titanic_test_data.csv')
test_ids = test_data['PassengerId']

train_data.head()
train_data.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [49]:
#Clean function


#drop unuseful columns
def clean(train_data):
    train_data = train_data.drop(['Ticket','Cabin', 'Name', "PassengerId"], axis = 1)

    #address NaN values
    cols = ['SibSp', 'Parch', 'Fare','Age']
    for col in cols:
        train_data[col].fillna(train_data[col].median(), inplace = True)

    train_data.Embarked.fillna("U", inplace = True)
    return train_data

train_data = clean(train_data)
test_data = clean(test_data)

    

In [50]:
train_data.head(5)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [51]:
test_data.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,male,34.5,0,0,7.8292,Q
1,3,female,47.0,1,0,7.0000,S
2,2,male,62.0,0,0,9.6875,Q
3,3,male,27.0,0,0,8.6625,S
4,3,female,22.0,1,1,12.2875,S


In [52]:
#map strings to integer values, preprocessing step 

from sklearn import preprocessing
le = preprocessing.LabelEncoder()

cols = ['Sex', 'Embarked']

for col in cols:
    train_data[col] = le.fit_transform(train_data[col])
    test_data[col] = le.fit_transform(test_data[col])
    print(le.classes_)

train_data.head(5)


['female' 'male']
['C' 'Q' 'S']


,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,1,22.0,1,0,7.2500,2
1,1,1,0,38.0,1,0,71.2833,0
2,1,3,0,26.0,0,0,7.9250,2
3,1,1,0,35.0,1,0,53.1000,2
4,0,3,1,35.0,0,0,8.0500,2


In [53]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

y = train_data['Survived']
X = train_data.drop('Survived', axis=1)

X_train,X_test,y_train,y_test = train_test_split(X,y, test_size = 0.2, random_state=42)

In [54]:
#Train classifier
clf = LogisticRegression(random_state=0, max_iter = 1000).fit(X_train, y_train)

In [55]:
predictions = clf.predict(X_test)
from sklearn.metrics import accuracy_score
accuracy_score(y_test, predictions)

0.8100558659217877

In [56]:
submission_preds = clf.predict(test_data)

In [57]:
#Create file to upload to kaggle
df = pd.DataFrame({'PassengerId': test_ids.values,
                    'Survived': submission_preds,
                    })

In [58]:
df.to_csv('submission.csv', index=False)